In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf

import os
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print("Running on TPU ", tpu.cluster_spec().as_dict()["worker"])
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    print("Not connected to a TPU runtime. Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()

In [ ]:
strategy.scope()

In [ ]:
# !conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
# !conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
# !conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
# !conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
# !conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
# !conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
# !pip install efficientnet

In [ ]:
# import efficientnet.keras as efn
import os
import shutil
import random
import tensorflow as tf
from tensorflow.keras.layers import Dense , Conv2D , Dropout , MaxPooling2D , Flatten, Activation , BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.optimizers import Adam, SGD, Adadelta
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
random.seed(7)

In [ ]:
preprocess_input = tf.keras.applications.efficientnet.preprocess_input


In [ ]:
# base_model = EfficientNetB7(weights='imagenet',include_top=False,input_shape=(600,600,3))
# base_model.trainable = False
# base_model.summary()

In [ ]:
%cd ..
%mkdir tmp
%cd tmp
%ls


In [ ]:
a = os.listdir('../input/efficientnetb7-dataset-augmented/augmented/train')
a

In [ ]:
# for i in a:
#     os.mkdir('NewPreprocessed/{}'.format(i))

In [ ]:
shutil.copytree("../input/efficientnetb7-dataset-augmented/augmented/train","/kaggle/tmp/NewPreprocessed")

In [ ]:
# from PIL import Image
# import os, sys


# def resize(path,savepath):
#     for item in os.listdir(path):
#         item_path = path+"/"+item
#         im = Image.open(item_path)
#         imResize = im.resize((600,600), Image.ANTIALIAS)
#         imResize.save(savepath +"/"+item.split('.')[0]+ '_resized.jpg', 'JPEG')

# for i in a:
#     path = "../input/rsna-balenced-dataset/Processed/"+i
#     savepath = "NewPreprocessed/"+i
#     resize(path,savepath)
    

In [ ]:
%ls

In [ ]:
for i in a:
    savepath = "NewPreprocessed/"
    print(len(os.listdir(savepath+i)))

In [ ]:
split_size = .85
def split_data(SOURCE, TRAINING, VALIDATION, SPLIT_SIZE):  
    all_images = os.listdir(SOURCE)
    print(type(all_images))
    random.shuffle(all_images)
    splitting_index = round(SPLIT_SIZE*len(all_images))

    #print(splitting_index)
    #print(splitting_index+portion)

    train_images = all_images[:splitting_index]
    valid_images = all_images[splitting_index:]
    for img in train_images:
        shutil.copy(os.path.join(SOURCE,img),TRAINING)
    for img in valid_images:
        shutil.copy(os.path.join(SOURCE,img),VALIDATION)


In [ ]:
os.mkdir('train')
os.mkdir('val')

In [ ]:
for i in a:
    source = "NewPreprocessed/"+i
    train = "train/"+i
    val = "val/"+i
    os.mkdir(train)
    os.mkdir(val)
    split_data(source, train, val, split_size)

In [ ]:
for i in a:
    train = "train/"+i
    val = "val/"+i
    print(len(os.listdir(train)),len(os.listdir(val)))

In [ ]:
batch_size = 64


In [ ]:
# TRAINING_DIR = 'train'
# train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
# train_generator =  train_datagen.flow_from_directory(TRAINING_DIR,
#                                                       target_size=(600,600),
#                                                       batch_size=16,
#                                                      class_mode='categorical',
#                                                      shuffle = True)
# VALIDATION_DIR = 'val'
# validation_datagen = ImageDataGenerator()
# validation_generator =  validation_datagen.flow_from_directory(VALIDATION_DIR,
#                                                       target_size=(600,600),
#                                                       batch_size=16,
#                                                       class_mode='categorical',
#                                                       shuffle = True)

In [ ]:
TRAINING_DIR = 'train'
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
                                                      TRAINING_DIR,
                                                        shuffle = True,
                                                      image_size=(600,600),
                                                      batch_size=batch_size,
                                                        label_mode='categorical')
VALIDATION_DIR = 'val'
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
                                                      VALIDATION_DIR,
                                                    shuffle = True,
                                                      image_size=(600,600),
                                                    label_mode='categorical',
                                                      batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
class_names

In [ ]:
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_names[labels[i]])
#         plt.axis("off")

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
# base_model = EfficientNetB7(weights='imagenet',include_top=False,input_shape=(600,600,3))

In [ ]:
# base_model.trainable = False
# for i in base_model.layers[-10:]:
#     i.trainable = True
#     print(i.trainable)
# for i in base_model.layers[-20:]:
#     print(i.trainable)

In [ ]:
def create_model():
    inputs = tf.keras.Input(shape=(600, 600, 3))
    preprocess_input = tf.keras.applications.efficientnet.preprocess_input
    data_augmentation = tf.keras.Sequential([
                                              #tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
                                              tf.keras.layers.experimental.preprocessing.RandomRotation(0.03),
                                            ])
    
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
#     prediction_layer0 = tf.keras.layers.Dense(2048,activation="relu")
#     prediction_layer1 = tf.keras.layers.Dense(1024,activation="relu")
    prediction_layer2 = tf.keras.layers.Dense(512,activation="relu")
    prediction_layer3 = tf.keras.layers.Dense(128,activation="relu")
    prediction_layer4 = tf.keras.layers.Dense(4,activation="softmax")
    
    #x = data_augmentation(inputs)
    x = preprocess_input(inputs)
    base_model = EfficientNetB7(weights='imagenet',include_top=False,input_tensor=x)
    base_model.trainable = False
    for layer in base_model.layers[-10:]:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True
    x = global_average_layer(base_model.output)
#     x = BatchNormalization()(x)
#     x = Dropout(0.2)(x)
#     x = prediction_layer0(x)
#     x = Dropout(0.2)(x)
#     x = prediction_layer1(x)
    x = Dropout(0.1)(x)
    x = prediction_layer2(x)
    x = Dropout(0.2)(x)
    x = prediction_layer3(x)
    outputs = prediction_layer4(x)
    model = tf.keras.Model(inputs, outputs)
    return(model)

In [ ]:
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=5000,
    decay_rate=0.1,
    staircase=False)


In [ ]:
with strategy.scope():
    Model = create_model()
    optimizer = Adam(learning_rate=lr_schedule)
    Model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
#Model = tf.keras.models.load_model("../input/efficientnetb7/weights/my_model.h5")

history = Model.fit(train_ds,epochs=50,batch_size=batch_size,validation_data=val_ds) 

Model.save('../working/weights/my_model.h5')

In [ ]:
#Model.save('../working/weights/my_model.h5')

In [ ]:
# with strategy.scope():
#     Model = create_model()
#     optimizer = SGD(learning_rate=lr_schedule)
#     Model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
# Model = tf.keras.models.load_model("../input/efficientnetb7/weights/my_model.h5")

# history = Model.fit(train_ds,epochs=10,batch_size=batch_size,validation_data=val_ds) 

# Model.save('../working/weights/my_model.h5')

In [ ]:
# with strategy.scope():
#     Model = create_model()
#     optimizer = Adadelta(learning_rate=lr_schedule)
#     Model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
# Model = tf.keras.models.load_model("../input/efficientnetb7/weights/my_model.h5")

# history = Model.fit(train_ds,epochs=6,batch_size=batch_size,validation_data=val_ds) 

# Model.save('../working/weights/my_model.h5')

In [ ]:
print(Model.input_shape,Model.output_shape)
#Model.summary()

In [ ]:

# history = Model.fit(train_generator,epochs=40,batch_size=16,validation_data=validation_generator)
#print(len(base_model.layers))

In [ ]:
# with strategy.scope():

In [ ]:
# os.mkdir("../working/weights")
# Model.save_weights("../working/weights/weights.pt")

Loading and saving

In [ ]:
# os.rmdir("../working/weights")

In [ ]:
#Model = tf.keras.models.load_model("../input/efficientnetb7/weights/my_model.h5")

In [ ]:
#history = Model.fit(train_ds,epochs=25,batch_size=16,validation_data=val_ds) 

In [ ]:
#Model =  tf.keras.models.load_model("/kaggle/input/efficientnetweights/weights/my_model.h5")

In [ ]:
#Model = tf.keras.models.load_model("/kaggle/input/efnetb7-layers-increased-more-trainable-layers/weights/my_model.h5")

In [ ]:
# Model = create_model()
#Model.load_weights("../input/efficientnetweights/weights/")

In [ ]:
#Model.evaluate(val_ds)

In [ ]:
# def unfreeze_model(Model):
#     # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
#     for layer in Model.layers[-30:]:
#         if not isinstance(layer, tf.keras.layers.BatchNormalization):
#             layer.trainable = True

#     optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
#     Model.compile(
#         optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
#     )


# unfreeze_model(Model)

# epochs = 10  # @param {type: "slider", min:8, max:50}
# hist = Model.fit(train_ds, epochs=epochs,batch_size=batch_size, validation_data=val_ds)
# plot_hist(hist)

In [ ]:
#Model.summary()

In [ ]:
#Model.layers